## Step 2: Micov Filter + Combine Technical Replicates
**Goal: Run Micov, combine technical replicates and export the files**

Github: https://github.com/biocore/micov

Conda: https: comad_EAC_figures, terminal: comad

### Imports

In [1]:
import pandas as pd
from biom import load_table
from qiime2 import Artifact

from qiime2.plugins.feature_table.methods import filter_features
from python_scripts.micov_filter import *

In [2]:
#Import biom files from SCRuB into pandas df (qza was in differnt qiime2 format [like an older verison] so couldn't go q2->df)

# Load the BIOM table
biom_table = load_table('processed_data/SCRuB/pese_pangenome_align-RS210_masked_none_scrubbed.qza_extractBiom/data/feature-table.biom')

# Convert the BIOM table to a pandas DataFrame
df_rs210c = biom_table.to_dataframe(dense=True)

df_rs210c

,HCC11.B.S137.L003.aligned,CRC9.B.2.S94.L003.aligned,HCC.3.P.S115.L003.aligned,CRC7.T.S87.L003.aligned,CRC2.B.S65.L003.aligned,CRC1.T.2.S56.L003.aligned,CRC10.T.2.S60.L003.aligned,CRC11.T.S63.L003.aligned,HCC.3.P.2.S116.L003.aligned,HCC13.T.S147.L003.aligned,...,HCC16.B.S157.L003.aligned,HCC12.B.S141.L003.aligned,HCC.5.P.2.S120.L003.aligned,CRC.8.P.S49.L003.aligned,HCC10.T.S135.L003.aligned,HCC16.T.2.S160.L003.aligned,CRC10.T.S59.L003.aligned,CRC6.T.2.S84.L003.aligned,HCC.10.P.2.S100.L003.aligned,CRC5.B.2.S78.L003.aligned
G000001985,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
G000002415,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,2.0
G000002445,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
G000002495,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
G000002655,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
G907163265,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
G907166805,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
G910573815,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
G910576415,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
#Metadata import
meta = pd.read_csv('qiita_downloads/pese_pangenome_align-RS210-masked/meta_pese_pangenome_align-rs210-masked.tsv', sep='\t')
meta[:2]

,sample_name,empo_1,empo_2,empo_3,empo_4,country,latitude,taxon_id,elevation,env_biome,...,run_center,run_date,run_prefix,runid,sample_plate,sample_project,sample_well,sequencing_meth,well_description,sampleid
0,15336.BLANK1.8D.2,Control,Negative,Sterile water blank,Sterile water blank,USA,32.5,256318,193,urban biome,...,IGM,4/18/21,BLANK1_8D_2_S12_L003,210418_A00953_0278_BH5G5VDSX2,HCC_Tissue_Plasma_P1.2,HCC_Tissue_Plasma,H8,sequencing by synthesis,HCC_Tissue_Plasma_P1.2.BLANK1.8D.2.H8,BLANK1.8D.2.S12.L003.aligned
1,15336.BLANK1.8E.2,Control,Negative,Sterile water blank,Sterile water blank,USA,32.5,256318,193,urban biome,...,IGM,4/18/21,BLANK1_8E_2_S14_L003,210418_A00953_0278_BH5G5VDSX2,HCC_Tissue_Plasma_P1.2,HCC_Tissue_Plasma,J8,sequencing by synthesis,HCC_Tissue_Plasma_P1.2.BLANK1.8E.2.J8,BLANK1.8E.2.S14.L003.aligned


### Functions

In [4]:
def combo_tech_rep(df, fn, micov_filter=False):
    
    #Rename headers to match meta
    mapping = dict(zip(meta['sampleid'], meta['sample_name']))

    # Rename columns in df using the mapping
    df.rename(columns=mapping, inplace=True)
    
    #Find cols ending with '.2'
    rep_cols = [col for col in df.columns if col.endswith('.2')]

    #Create new df with paired technical replicates
    df_techReps = pd.DataFrame()
    for col in rep_cols:
        col_name = col[:-2]  #remove the '.2'
        if col_name in df.columns:
            df_techReps[col_name] = df[col_name] + df[col]  
            
    if micov_filter != False:
        df_techReps = micov_pass_fail(df_techReps)
            
    #Export subset df as qza
    qza = Artifact.import_data("FeatureTable[Frequency]", df_techReps.T)
    
    qza.save('processed_data/TechRep_merged/'+ fn + '.qza')

    return(df_techReps)

In [5]:
def micov_pass_fail(df):
    '''Taking micov pass_ and fail_ lists - subset dataset'''
    
    #Import pass and fail lists
    pass_ = list(pd.read_csv('micov/micov_filtered_lists/PASSmicov.csv', sep='\t')['Pass'])
    fail_ = list(pd.read_csv('micov/micov_filtered_lists/FAILmicov.csv', sep='\t')['Fail'])
    
    #Check to make sure none are missed
    for i in df.index:
        if i not in pass_ and i not in fail_:
            print(i)
    
    subset_df = df.loc[pass_]
    
    return(subset_df)
    

In [6]:
def save_pass_fail(pass_, fail_):
    '''Save pass and fail lists for future use'''
    
    #pd.DataFrame(pass_, columns=['Pass'])['Pass'].to_csv('micov/micov_filtered_lists/PASSmicov.csv', sep='\t', index=False)
    #pd.DataFrame(fail_, columns=['Fail'])['Fail'].to_csv('micov/micov_filtered_lists/FAILmicov.csv', sep='\t', index=False)
    #A bit repetative to save here and upload there but wanted to make sure we had clear copy of list
    
    print('already have so not re-running')

### Determine MissMapped Microbes from Micov

In [7]:
taxa = choose_taxa('RS210')

#Import dataset
gen_len = pd.read_csv('micov/all_microbes.coverage', sep='\t')
pos = pd.read_csv('micov/all_microbes.covered_positions', sep='\t')
 
#Import biom table from prior to SCRub becuase Micov run at the SAM level while SCRuB doesn't 
micovRef_table = biom.load_table('qiita_downloads/pese_pangenome_align-RS210-masked/biom/none.biom')
micovRef_df = micovRef_table.to_dataframe(dense=True)

cov= pd.read_csv('micov/coverage_percentage.txt', sep='\t')
cov = cov.rename(columns={'genome_id': 'GOTU', 'percent_covered':'coverage_percent'})

In [9]:
pass_, fail_ = loop_micov(gen_len, pos, micovRef_df, cov, taxa, plot=True)

G000001985 d__Eukaryota; p__Ascomycota; c__Eurotiomycetes; o__Eurotiales; f__Trichocomaceae; g__Talaromyces; s__Talaromyces marneffei
Coverage Percentage: 0.0008585522888881%
Status: Fail -- less than 5 BIOM hits
----------------------------
Length of pass 0
Length of fail 1
G000002415 d__Eukaryota; p__Apicomplexa; c__Aconoidasida; o__Haemosporida; f__Plasmodiidae; g__Plasmodium; s__Plasmodium vivax
Coverage Percentage: 0.000424846014562%
Status: Fail -- less than 30 reads
----------------------------
Length of pass 0
Length of fail 2
G000002445 d__Eukaryota; p__Euglenozoa; c__Kinetoplastea; o__Trypanosomatida; f__Trypanosomatidae; g__Trypanosoma; s__Trypanosoma brucei
Coverage Percentage: 0.0018753081890681%
Adj, read counts 66
Number of samples per group [1, 1, 54, 10]
Number of bins 4
total # samples 56
Number of total reads 66
Number of hits in BIOM tables 26.0

Percentage of reads in the top 3 bin groups with the largest samples 98.48484848484848 %
Percentage of reads from samples

/Users/cguccion/Dropbox/current_Fall23/HCC_Amir_v1/HCC-microbialDNA/python_scripts/micov_filter.py:68: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


G000002725 d__Eukaryota; p__Euglenozoa; c__Kinetoplastea; o__Trypanosomatida; f__Trypanosomatidae; g__Leishmania; s__Leishmania major
Coverage Percentage: 0.0026509787970698%
Status: Fail -- less than 5 BIOM hits
----------------------------
Length of pass 0
Length of fail 6
G000002765 d__Eukaryota; p__Apicomplexa; c__Aconoidasida; o__Haemosporida; f__Plasmodiidae; g__Plasmodium; s__Plasmodium falciparum
Coverage Percentage: 0.0032065637502597%
Status: Fail -- less than 5 BIOM hits
----------------------------
Length of pass 0
Length of fail 7
G000002825 d__Eukaryota; p__Parabasalia; c__; o__Trichomonadida; f__Trichomonadidae; g__Trichomonas; s__Trichomonas vaginalis
Coverage Percentage: 0.0004000778210445%
Status: Fail -- less than 5 BIOM hits
----------------------------
Length of pass 0
Length of fail 8
G000002845 d__Eukaryota; p__Euglenozoa; c__Kinetoplastea; o__Trypanosomatida; f__Trypanosomatidae; g__Leishmania; s__Leishmania braziliensis
Coverage Percentage: 0.0030338452092263%


/Users/cguccion/Dropbox/current_Fall23/HCC_Amir_v1/HCC-microbialDNA/python_scripts/micov_filter.py:68: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


Status: Pass
----------------------------
Length of pass 1
Length of fail 11
G000006355 d__Eukaryota; p__Apicomplexa; c__Aconoidasida; o__Haemosporida; f__Plasmodiidae; g__Plasmodium; s__Plasmodium knowlesi
Coverage Percentage: 0.0044181705831071%
Status: Fail -- less than 30 reads
----------------------------
Length of pass 1
Length of fail 12
G000006405 d__Eukaryota; p__Perkinsozoa; c__; o__Perkinsida; f__Perkinsidae; g__Perkinsus; s__Perkinsus marinus
Coverage Percentage: 0.0004588158498774%
Status: Fail -- less than 5 BIOM hits
----------------------------
Length of pass 1
Length of fail 13
G000006445 d__Eukaryota; p__Ascomycota; c__Saccharomycetes; o__Saccharomycetales; f__Debaryomycetaceae; g__Debaryomyces; s__Debaryomyces hansenii
Coverage Percentage: 0.0040961779294321%
Status: Fail -- less than 30 reads
----------------------------
Length of pass 1
Length of fail 14
G000006765 d__Bacteria; p__Proteobacteria; c__Gammaproteobacteria; o__Pseudomonadales; f__Pseudomonadaceae; g__P

...

/Users/cguccion/Dropbox/current_Fall23/HCC_Amir_v1/HCC-microbialDNA/python_scripts/micov_filter.py:68: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
/Users/cguccion/Dropbox/current_Fall23/HCC_Amir_v1/HCC-microbialDNA/python_scripts/micov_filter.py:68: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
/Users/cguccion/Dropbox/current_Fall23/HCC_Amir_v1/HCC-microbialDNA/python_scripts/micov_filter.py:68: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


G904846325 d__Bacteria; p__Proteobacteria; c__Gammaproteobacteria; o__Moraxellales; f__Moraxellaceae; g__Psychrobacter; s__Psychrobacter marincola
Coverage Percentage: 0.017156901992913%
Status: Fail -- less than 5 BIOM hits
----------------------------
Length of pass 466
Length of fail 3655
G904846345 d__Bacteria; p__Proteobacteria; c__Gammaproteobacteria; o__Moraxellales; f__Moraxellaceae; g__Psychrobacter; s__Psychrobacter maritimus
Coverage Percentage: 0.1887531338182907%
Adj, read counts 37
Number of samples per group [1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1]
Number of bins 31
total # samples 16
Number of total reads 37
Number of hits in BIOM tables 37.0

Percentage of reads in the top 3 bin groups with the largest samples 16.216216216216218 %
Percentage of reads from samples who only fall into one bin group 18.91891891891892 %
Status: Pass
----------------------------
Length of pass 467
Length of fail 3655
G904846355 d__Bacteria; p__P

/Users/cguccion/Dropbox/current_Fall23/HCC_Amir_v1/HCC-microbialDNA/python_scripts/micov_filter.py:68: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


G904846715 d__Bacteria; p__Proteobacteria; c__Gammaproteobacteria; o__Moraxellales; f__Moraxellaceae; g__Psychrobacter; s__Psychrobacter vallis
Coverage Percentage: 0.0220241837946055%
Status: Fail -- less than 5 BIOM hits
----------------------------
Length of pass 468
Length of fail 3659
G904848625 d__Bacteria; p__Proteobacteria; c__Betaproteobacteria; o__Burkholderiales; f__Burkholderiaceae; g__Paraburkholderia; s__Paraburkholderia metrosideri
Coverage Percentage: 0.0019251103826208%
Status: Fail -- less than 5 BIOM hits
----------------------------
Length of pass 468
Length of fail 3660
G905142465 d__Bacteria; p__Proteobacteria; c__Gammaproteobacteria; o__Xanthomonadales; f__Xanthomonadaceae; g__Xanthomonas; s__Xanthomonas hydrangea
Coverage Percentage: 0.0233870354280554%
Status: Fail -- less than 5 BIOM hits
----------------------------
Length of pass 468
Length of fail 3661
G905220975 d__Bacteria; p__Proteobacteria; c__Betaproteobacteria; o__Burkholderiales; f__Burkholderiaceae;

/Users/cguccion/Dropbox/current_Fall23/HCC_Amir_v1/HCC-microbialDNA/python_scripts/micov_filter.py:68: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [10]:
print(len(pass_))
len(fail_)

469


3672

In [14]:
save_pass_fail(pass_, fail_)

### Combine Tech Reps + Micov

In [8]:
#Input scrubed df, output with combined tech reps + micov
combo_tech_rep(df_rs210c, 'pese_pangenome_align-RS210_masked_none_scrubbed_micovFilter', micov_filter=True)

,15336.CRC.9.Background,15336.CRC.1.tumor,15336.CRC.10.tumor,15336.HCC.3.Plasma,15336.HCC.2.tumor,15336.HCC.1.Plasma,15336.HCC.12.Background,15336.HCC.14.tumor,15336.HCC.5.tumor,15336.CRC.5.tumor,...,15336.CRC.11.Background,15336.CRC.3.Plasma,15336.HCC.9.Background,15336.HCC.8.tumor,15336.CRC.9.Plasma,15336.HCC.5.Plasma,15336.HCC.16.tumor,15336.CRC.6.tumor,15336.HCC.10.Plasma,15336.CRC.5.Background
G000005845,0.0,1.0,0.0,0.0,0.0,0.0,0.0,20.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
G000006765,0.0,3.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,10.0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,3.0,0.0
G000006925,0.0,1.0,0.0,0.0,0.0,0.0,0.0,15.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
G000008865,0.0,1.0,0.0,0.0,0.0,0.0,0.0,17.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
G000010125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
G904063075,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0
G904846155,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,2.0,0.0,0.0,2.0,2.0,0.0,0.0,4.0,0.0
G904846345,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,3.0,0.0
G904846465,0.0,0.0,0.0,5.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,3.0,0.0,0.0,3.0,10.0,0.0,0.0,3.0,0.0
